In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#!ls "/content/gdrive/MyDrive/"

In [3]:
!git clone https://github.com/knamdar/data.git

Cloning into 'data'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 2), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [4]:
!pip install foolbox

     |████████████████████████████████| 1.7MB 8.7MB/s 
     |████████████████████████████████| 174kB 54.6MB/s 
     |████████████████████████████████| 61kB 11.0MB/s 
     |████████████████████████████████| 71kB 13.4MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import foolbox as fb
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD,LinfBasicIterativeAttack,LinfFastGradientAttack,L2CarliniWagnerAttack,LinfDeepFoolAttack,L2DeepFoolAttack,L2BasicIterativeAttack,L2ProjectedGradientDescentAttack

class Model_Drop(nn.Module):
    def __init__(self):

        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(7*7*64, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
        self.drop_layer = nn.Dropout(p=0.50)

    def last_hidden_layer_output(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        x = x.view(-1, 7*7*64)
        x = self.drop_layer((F.relu(self.fc1(x))))
        x = self.drop_layer((F.relu(self.fc2(x))))
        return x

    def forward(self, x):
        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x

mnist_train = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("data", train=False, download=True, transform=transforms.ToTensor())

train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=128, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

torch.manual_seed(2)
torch.cuda.manual_seed(2)

softmax = nn.Softmax(dim=1)

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

criterion = nn.CrossEntropyLoss()

T = 1

def epoch(loader, model, opt=None):

    if opt:
        model.train()
    else:
        model.eval()

    total_loss, total_err = 0., 0.

    for X, y in loader:
        X, y = X.to(device), y.to(device)
        
        outputs = model(X)

        log_prob_s = F.log_softmax(outputs / T, dim=1)

        loss = F.nll_loss(log_prob_s,y)
        #loss = criterion(log_prob_s, y)

        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()

        total_err += (outputs.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)



In [6]:
model_cnn = Model_Drop()
model_cnn.to(device)

opt = optim.Adam(model_cnn.parameters(), lr=0.001)
#opt = optim.SGD(model_cnn.parameters(), lr=0.1, momentum=0.5)

for te in range(10):


    train_err, train_loss = epoch(train_loader, model_cnn, opt)
    test_err, test_loss = epoch(test_loader, model_cnn) 

    print(*("{:.6f}".format(i) for i in (train_err, test_err)), sep="\t")




/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0.124100	0.019200
0.030100	0.011800
0.023117	0.010900
0.016617	0.010000
0.014400	0.008400
0.012167	0.008100
0.010983	0.006200
0.009483	0.007600
0.009150	0.006400
0.008100	0.008900


In [7]:
model_cnn.eval()
torch.save(model_cnn.state_dict(), "TENET_DIGIT_MNIST_model_cnn_normal.pt")

model_cnn.load_state_dict(torch.load("TENET_DIGIT_MNIST_model_cnn_normal.pt"))
model_cnn.eval()

Model_Drop(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
  (drop_layer): Dropout(p=0.5, inplace=False)
)

In [8]:
torch.save(model_cnn.state_dict(), "/content/gdrive/MyDrive/TENET_DIGIT_MNIST_model_cnn_normal.pt")